In [1]:
%load_ext autoreload
%autoreload 2
%load_ext lab_black

In [2]:
import pandas as pd
import numpy as np
import random
from scipy import stats
import admix_prs

In [3]:
# Constats
trait = "bmi"
PHENO_PATH = "/u/project/pasaniuc/kangchen/tmp/prs-1219/REAL-PHENO/all-pheno.csv"
PRS_PATH = f"/u/project/pasaniuc/kangchen/tmp/prs-1219/REAL-PRS/{trait}.tsv.gz"

In [4]:
df_pheno = pd.read_csv(PHENO_PATH, index_col=0)
df_pheno.index = df_pheno.index.astype(str)
df_prs = pd.read_csv(PRS_PATH, sep="\t")
df_prs["indiv"] = df_prs["indiv"].apply(lambda x: x.split("_")[0])
df_prs = df_prs.set_index("indiv")

In [5]:
df_info = pd.merge(df_pheno, df_prs, left_index=True, right_index=True)
df_info = df_info.dropna(subset=[trait, "MEAN"])

# Calculate R2 stratified by age and sex

In [6]:
res1 = admix_prs.stratify_calculate_r2(
    df_info, x_col="bmi", y_col="MEAN", stratify_col="SEX"
)

# TODO:
res2["AGE_Q"] = admix_prs.make_levels(res2["AGE"])

# either case 1
res2 = admix_prs.stratify_calculate_r2(
    df_info, x_col="bmi", y_col="MEAN", stratify_col="AGE_Q"
)

# either case 2
res2 = admix_prs.stratify_calculate_r2(
    df_info, x_col="bmi", y_col="MEAN"
)

# 1. use qcut
# 2. level header -> stratify_col
# 3. level values -> straity_col's values

In [7]:
res1

,level,R2,R2_std
0,1,0.241629,0.000988
1,2,0.221893,0.001304


In [8]:
res2

,level,R2,R2_std
0,1,0.234256,0.002806
1,2,0.240714,0.001812
2,3,0.236706,0.003211
3,4,0.229604,0.002346
4,5,0.222807,0.001538


# Evaluate calibration

In [9]:
res1 = admix_prs.eval_calibration(
    df_info,
    x_col="bmi",
    lower_col="QUANTILE_10",
    upper_col="QUANTILE_90",
    stratify_col="SEX",
)
res2 = admix_prs.eval_calibration(
    df_info,
    x_col="bmi",
    lower_col="QUANTILE_10",
    upper_col="QUANTILE_90",
    stratify_col="AGE",
)

In [10]:
res1

,level,coverage
0,1,0.0
1,2,0.0


In [11]:
res2

,level,coverage
0,1,0.0
1,2,0.0
2,3,0.0
3,4,0.0
4,5,0.0
